# LightHearted: Tutorial

This notebook presents a walkthrough of how to use the LightHearted framework to create a lighting design. For this, we use the design employed in a concert with the Aarhus Symphony Orchestra (ASO) for a performance of Tchaikovsky's Romeo and Juliet Fantasy Overture. The script containing this can be found in the examples folder . By the end of this tutorial you should hopefully be familiar with the use of the five main objects used in a LightHearted workflow:

- FIFOBuffer - Used to contain and process ECG signals
- LightingArray - Used to define a lighting group and send them commands
- MappingArray - Used to contain and reduce data to be mapped from multiple ECG devices and derive spatial expansions that can be mapped to LightingArrays
- ContinuousMapper - Used to define the relationship between a MappingArray and a LightingArray
- TriggerMapper - Used to define event based triggers between FIFOBuffers and LightingArrays

## 1. Basics

### 1.1 Workflow

The general workflow for LightHearted is as follows:

1. ECG signals are transported over OSC and received in a script. For each signal, a first-in-first-out (FIFO) buffer is instanced.
2. Optional chains of transforms are applied to each FIFOBuffer, and placed into new instances of FIFOBuffers (e.g. filtering, QRS extraction).
3. Signal or transformed buffers can be assigned to instances of MappingArrays. These take multiple FIFOBuffers, with their position in the array corresponding to a spatial position. Each buffer is reduced to a single value using a chain of functions (e.g. mean, the newest value in the array).
4. The MappingArray can be used to generate spatial expansions, that is expanding the MappingArray to match the shape of a LightingArray. This can be done through a user-defined function (e.g. linear interpolation, filling the expansion with set values). A single MappingArray can generate multiple spatial expansions if the data is to be used to generate lighting across multiple lighting groups.
5. Groups of lighting fixtures are defined as LightingArray objects. If interpolations are to be used to generate spatial expansions in the MappingArray, anchor positions in the LightingArray can also be defined. These correspond to the spatial positions of the values in the MappingArray before expansion.
6. ContinuousMappers can be instanced to define the relationship between the MappingArrays/spatial expansions and the LightingArrays. These result in the intensity and colour parameters of the LightingArray being continuously updated.
7. The LightingArray can be used to send commands to the installed lighting system in the concert hall, sending the corresponding parameter messages to the fixtures.
8. TriggerMapper objects can be defined to generate action based mappings between FIFOBuffers. These take one buffer as a reference and another as a query (e.g. a signal buffer could be a reference, and a buffer of peaks indices could be a query). A chain of trigger functions can be defined, as long as the final function returns a bool (e.g. has a peak index in the peak buffer crossed a specified index in the signal buffer). Upon return of a True value from the trigger function chain, an action function is triggered.

These should be implemented in a single script, encapsulated in async functions. Familiarity with the asyncio library is assumed. If there are many parameters to define, it is also recommeded to make use of a config script, which is imported into the main script.

### 1.2 The ASO Design

The ASO design consists of three primary lighting groups and corresponding mappings:

1. A group of 14 horizonally organised LEDs reflected from the organ directly behind the stage. For this, we derive heart rate values from the ECG signals, interpolate the values to match the number of LEDs, and use these to drive shifts in RGB values within a defined colourmap. We also use the detection of an R-Peak to trigger changes in the lighting intensity.
2. A group of 36 baluster leds, mounted in the balconies around the audience. For these, we use the derived heart rate from the conductor's ECG to generate shifts in RGB within the colourmap.
3. A horizontally arranged row of 15 background LEDs, mounted in the wall to the rear of the stage. We use these to display the current colourmap.

A video of the design in action can be found [here](https://osf.io/c2zt9/?view_only=23cc8068eba347b2a7cc4f6dbc77adc3).

# 2. First Steps

## 2.1 Setting Up a Script

The first step is to create a new script, which we will call ASO25.py. At first, we will just import asyncio, used to define our async functions.

In [ ]:
import asyncio

We will then define a main function, and set it to run.

In [ ]:
async def main():
    pass

if __name__ == "__main__":
    asyncio.run(main())

We will define separate processing functions, and these will be launched from the main function. We will also initialise all of our LightHearted objects here.